# Day 01: Markowitz Mean-Variance Optimization

## Week 18: Portfolio Optimization

### Learning Objectives
- Understand Modern Portfolio Theory (MPT) fundamentals
- Implement mean-variance optimization from scratch
- Construct and visualize the efficient frontier
- Calculate key portfolio statistics (Sharpe ratio, volatility, returns)
- Apply optimization to real market data using yfinance

### Topics Covered
1. Portfolio Theory Fundamentals
2. Expected Returns & Covariance Estimation
3. Mean-Variance Optimization
4. Efficient Frontier Construction
5. Portfolio Statistics & Risk Metrics
6. Practical Implementation with Real Data

---
## 1. Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Set random seed for reproducibility
np.random.seed(42)

print("Setup complete!")

---
## 2. Theory: Modern Portfolio Theory (MPT)

### 2.1 Key Concepts

**Harry Markowitz (1952)** introduced Modern Portfolio Theory, which revolutionized investment management.

**Core Principles:**
- Investors are risk-averse (prefer less risk for same return)
- Diversification reduces portfolio risk
- Optimal portfolios lie on the "efficient frontier"

### 2.2 Mathematical Framework

**Portfolio Return:**
$$R_p = \sum_{i=1}^{n} w_i R_i = \mathbf{w}^T \mathbf{\mu}$$

**Portfolio Variance:**
$$\sigma_p^2 = \sum_{i=1}^{n} \sum_{j=1}^{n} w_i w_j \sigma_{ij} = \mathbf{w}^T \mathbf{\Sigma} \mathbf{w}$$

**Where:**
- $w_i$ = weight of asset $i$
- $R_i$ = return of asset $i$
- $\mu$ = vector of expected returns
- $\Sigma$ = covariance matrix

### 2.3 Optimization Problem

**Minimize variance for a target return:**
$$\min_{\mathbf{w}} \mathbf{w}^T \mathbf{\Sigma} \mathbf{w}$$

**Subject to:**
- $\mathbf{w}^T \mathbf{\mu} = \mu_{target}$ (target return)
- $\mathbf{w}^T \mathbf{1} = 1$ (weights sum to 1)
- $w_i \geq 0$ (no short selling, optional)

---
## 3. Data Collection with yfinance

In [ ]:
# Define portfolio assets - diversified across sectors
tickers = [
    'AAPL',   # Technology
    'MSFT',   # Technology
    'JNJ',    # Healthcare
    'JPM',    # Financials
    'XOM',    # Energy
    'PG',     # Consumer Staples
    'AMZN',   # Consumer Discretionary
    'GLD',    # Gold ETF (alternative)
]

# Download historical data
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)  # 5 years of data

print(f"Downloading data from {start_date.date()} to {end_date.date()}...")
data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']

# Display data info
print(f"\nData shape: {data.shape}")
print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")
print(f"\nAssets: {list(data.columns)}")
data.tail()

In [ ]:
# Calculate daily returns
returns = data.pct_change().dropna()

print(f"Returns shape: {returns.shape}")
print(f"\nReturns Statistics:")
returns.describe().round(4)

In [ ]:
# Visualize price evolution (normalized)
normalized_prices = data / data.iloc[0] * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Normalized prices
normalized_prices.plot(ax=axes[0], linewidth=1.5)
axes[0].set_title('Normalized Asset Prices (Base = 100)', fontsize=12)
axes[0].set_ylabel('Normalized Price')
axes[0].legend(loc='upper left', fontsize=8)

# Returns distribution
returns.boxplot(ax=axes[1])
axes[1].set_title('Daily Returns Distribution by Asset', fontsize=12)
axes[1].set_ylabel('Daily Return')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---
## 4. Expected Returns and Covariance Estimation

In [ ]:
# Trading days per year
TRADING_DAYS = 252

# Calculate annualized expected returns (mean historical returns)
expected_returns = returns.mean() * TRADING_DAYS

# Calculate annualized covariance matrix
cov_matrix = returns.cov() * TRADING_DAYS

# Calculate correlation matrix
corr_matrix = returns.corr()

print("Annualized Expected Returns:")
print(expected_returns.round(4).to_string())
print(f"\nExpected Return Range: {expected_returns.min():.2%} to {expected_returns.max():.2%}")

In [ ]:
# Visualize covariance and correlation matrices
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Correlation heatmap
sns.heatmap(corr_matrix, annot=True, cmap='RdYlGn', center=0, 
            fmt='.2f', ax=axes[0], vmin=-1, vmax=1)
axes[0].set_title('Correlation Matrix', fontsize=12)

# Covariance heatmap
sns.heatmap(cov_matrix, annot=True, cmap='YlOrRd', 
            fmt='.4f', ax=axes[1])
axes[1].set_title('Annualized Covariance Matrix', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Individual asset risk-return profile
individual_volatility = np.sqrt(np.diag(cov_matrix))

risk_return_df = pd.DataFrame({
    'Asset': tickers,
    'Expected Return': expected_returns.values,
    'Volatility': individual_volatility,
    'Sharpe Ratio': expected_returns.values / individual_volatility  # Assuming rf = 0
})

print("Individual Asset Risk-Return Profile:")
print(risk_return_df.round(4).to_string(index=False))

---
## 5. Portfolio Statistics Functions

In [ ]:
def portfolio_return(weights, expected_returns):
    """
    Calculate portfolio expected return.
    
    Parameters:
    -----------
    weights : array-like
        Portfolio weights
    expected_returns : array-like
        Expected returns for each asset
    
    Returns:
    --------
    float : Portfolio expected return
    """
    return np.dot(weights, expected_returns)


def portfolio_volatility(weights, cov_matrix):
    """
    Calculate portfolio volatility (standard deviation).
    
    Parameters:
    -----------
    weights : array-like
        Portfolio weights
    cov_matrix : array-like
        Covariance matrix of returns
    
    Returns:
    --------
    float : Portfolio volatility
    """
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))


def portfolio_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate=0.02):
    """
    Calculate portfolio Sharpe ratio.
    
    Parameters:
    -----------
    weights : array-like
        Portfolio weights
    expected_returns : array-like
        Expected returns for each asset
    cov_matrix : array-like
        Covariance matrix of returns
    risk_free_rate : float
        Risk-free rate (annualized)
    
    Returns:
    --------
    float : Sharpe ratio
    """
    ret = portfolio_return(weights, expected_returns)
    vol = portfolio_volatility(weights, cov_matrix)
    return (ret - risk_free_rate) / vol


def portfolio_stats(weights, expected_returns, cov_matrix, risk_free_rate=0.02):
    """
    Calculate all portfolio statistics.
    
    Returns:
    --------
    dict : Dictionary with return, volatility, and Sharpe ratio
    """
    ret = portfolio_return(weights, expected_returns)
    vol = portfolio_volatility(weights, cov_matrix)
    sharpe = (ret - risk_free_rate) / vol
    
    return {
        'return': ret,
        'volatility': vol,
        'sharpe_ratio': sharpe
    }

print("Portfolio statistics functions defined!")

In [ ]:
# Test with equal-weighted portfolio
n_assets = len(tickers)
equal_weights = np.array([1/n_assets] * n_assets)

eq_stats = portfolio_stats(equal_weights, expected_returns.values, cov_matrix.values)

print("Equal-Weighted Portfolio Statistics:")
print(f"  Weights: {equal_weights.round(3)}")
print(f"  Expected Return: {eq_stats['return']:.2%}")
print(f"  Volatility: {eq_stats['volatility']:.2%}")
print(f"  Sharpe Ratio: {eq_stats['sharpe_ratio']:.3f}")

---
## 6. Mean-Variance Optimization

In [ ]:
def minimize_volatility(target_return, expected_returns, cov_matrix, 
                        allow_short_selling=False):
    """
    Find minimum volatility portfolio for a target return.
    
    Parameters:
    -----------
    target_return : float
        Target portfolio return
    expected_returns : array-like
        Expected returns for each asset
    cov_matrix : array-like
        Covariance matrix
    allow_short_selling : bool
        Whether to allow negative weights
    
    Returns:
    --------
    dict : Optimization result with weights and portfolio stats
    """
    n_assets = len(expected_returns)
    
    # Initial guess (equal weights)
    init_weights = np.array([1/n_assets] * n_assets)
    
    # Constraints
    constraints = [
        # Weights sum to 1
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        # Target return constraint
        {'type': 'eq', 'fun': lambda w: portfolio_return(w, expected_returns) - target_return}
    ]
    
    # Bounds
    if allow_short_selling:
        bounds = tuple((-1, 1) for _ in range(n_assets))
    else:
        bounds = tuple((0, 1) for _ in range(n_assets))
    
    # Optimize
    result = minimize(
        fun=lambda w: portfolio_volatility(w, cov_matrix),
        x0=init_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    if result.success:
        weights = result.x
        stats = portfolio_stats(weights, expected_returns, cov_matrix)
        return {
            'weights': weights,
            'return': stats['return'],
            'volatility': stats['volatility'],
            'sharpe_ratio': stats['sharpe_ratio'],
            'success': True
        }
    else:
        return {'success': False, 'message': result.message}

print("Optimization function defined!")

In [ ]:
def maximize_sharpe_ratio(expected_returns, cov_matrix, risk_free_rate=0.02,
                          allow_short_selling=False):
    """
    Find the maximum Sharpe ratio portfolio (tangency portfolio).
    
    Returns:
    --------
    dict : Optimization result with weights and portfolio stats
    """
    n_assets = len(expected_returns)
    
    # Objective: minimize negative Sharpe ratio
    def neg_sharpe(weights):
        return -portfolio_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate)
    
    # Initial guess
    init_weights = np.array([1/n_assets] * n_assets)
    
    # Constraints
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    
    # Bounds
    if allow_short_selling:
        bounds = tuple((-1, 1) for _ in range(n_assets))
    else:
        bounds = tuple((0, 1) for _ in range(n_assets))
    
    # Optimize
    result = minimize(
        fun=neg_sharpe,
        x0=init_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    if result.success:
        weights = result.x
        stats = portfolio_stats(weights, expected_returns, cov_matrix, risk_free_rate)
        return {
            'weights': weights,
            'return': stats['return'],
            'volatility': stats['volatility'],
            'sharpe_ratio': stats['sharpe_ratio'],
            'success': True
        }
    else:
        return {'success': False, 'message': result.message}

print("Max Sharpe function defined!")

In [ ]:
def minimum_variance_portfolio(expected_returns, cov_matrix, allow_short_selling=False):
    """
    Find the global minimum variance portfolio.
    
    Returns:
    --------
    dict : Optimization result with weights and portfolio stats
    """
    n_assets = len(expected_returns)
    
    # Initial guess
    init_weights = np.array([1/n_assets] * n_assets)
    
    # Constraints: weights sum to 1
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    
    # Bounds
    if allow_short_selling:
        bounds = tuple((-1, 1) for _ in range(n_assets))
    else:
        bounds = tuple((0, 1) for _ in range(n_assets))
    
    # Optimize
    result = minimize(
        fun=lambda w: portfolio_volatility(w, cov_matrix),
        x0=init_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    if result.success:
        weights = result.x
        stats = portfolio_stats(weights, expected_returns, cov_matrix)
        return {
            'weights': weights,
            'return': stats['return'],
            'volatility': stats['volatility'],
            'sharpe_ratio': stats['sharpe_ratio'],
            'success': True
        }
    else:
        return {'success': False, 'message': result.message}

print("Minimum variance function defined!")

---
## 7. Find Optimal Portfolios

In [ ]:
# Convert to numpy arrays
mu = expected_returns.values
cov = cov_matrix.values

# Find minimum variance portfolio
min_var = minimum_variance_portfolio(mu, cov)

print("=" * 60)
print("MINIMUM VARIANCE PORTFOLIO")
print("=" * 60)
print(f"Expected Return: {min_var['return']:.2%}")
print(f"Volatility: {min_var['volatility']:.2%}")
print(f"Sharpe Ratio: {min_var['sharpe_ratio']:.3f}")
print("\nWeights:")
for ticker, weight in zip(tickers, min_var['weights']):
    if weight > 0.001:  # Only show significant weights
        print(f"  {ticker}: {weight:.2%}")

In [ ]:
# Find maximum Sharpe ratio portfolio
max_sharpe = maximize_sharpe_ratio(mu, cov, risk_free_rate=0.02)

print("=" * 60)
print("MAXIMUM SHARPE RATIO PORTFOLIO (Tangency Portfolio)")
print("=" * 60)
print(f"Expected Return: {max_sharpe['return']:.2%}")
print(f"Volatility: {max_sharpe['volatility']:.2%}")
print(f"Sharpe Ratio: {max_sharpe['sharpe_ratio']:.3f}")
print("\nWeights:")
for ticker, weight in zip(tickers, max_sharpe['weights']):
    if weight > 0.001:
        print(f"  {ticker}: {weight:.2%}")

---
## 8. Efficient Frontier Construction

In [ ]:
def compute_efficient_frontier(expected_returns, cov_matrix, n_points=100, 
                               allow_short_selling=False):
    """
    Compute the efficient frontier by finding minimum variance portfolios
    for different target returns.
    
    Returns:
    --------
    DataFrame : Efficient frontier portfolios with returns, volatilities, and weights
    """
    # Get min and max possible returns
    min_ret = expected_returns.min()
    max_ret = expected_returns.max()
    
    # Generate target returns
    target_returns = np.linspace(min_ret, max_ret, n_points)
    
    # Find efficient portfolios
    efficient_portfolios = []
    
    for target in target_returns:
        result = minimize_volatility(target, expected_returns, cov_matrix, 
                                     allow_short_selling)
        if result['success']:
            efficient_portfolios.append({
                'return': result['return'],
                'volatility': result['volatility'],
                'sharpe_ratio': result['sharpe_ratio'],
                'weights': result['weights']
            })
    
    return pd.DataFrame(efficient_portfolios)

print("Computing efficient frontier...")
efficient_frontier = compute_efficient_frontier(mu, cov, n_points=50)
print(f"Found {len(efficient_frontier)} efficient portfolios")
efficient_frontier.head()

In [ ]:
# Generate random portfolios for comparison
def generate_random_portfolios(expected_returns, cov_matrix, n_portfolios=5000):
    """
    Generate random portfolio allocations for visualization.
    """
    n_assets = len(expected_returns)
    results = []
    
    for _ in range(n_portfolios):
        # Random weights that sum to 1
        weights = np.random.random(n_assets)
        weights /= weights.sum()
        
        stats = portfolio_stats(weights, expected_returns, cov_matrix)
        results.append(stats)
    
    return pd.DataFrame(results)

print("Generating random portfolios...")
random_portfolios = generate_random_portfolios(mu, cov)
print(f"Generated {len(random_portfolios)} random portfolios")

In [ ]:
# Visualize the Efficient Frontier
fig, ax = plt.subplots(figsize=(12, 8))

# Plot random portfolios
scatter = ax.scatter(
    random_portfolios['volatility'], 
    random_portfolios['return'],
    c=random_portfolios['sharpe_ratio'],
    cmap='viridis',
    alpha=0.3,
    s=10,
    label='Random Portfolios'
)
plt.colorbar(scatter, label='Sharpe Ratio')

# Plot efficient frontier
ax.plot(
    efficient_frontier['volatility'], 
    efficient_frontier['return'],
    'r-',
    linewidth=3,
    label='Efficient Frontier'
)

# Plot individual assets
for i, ticker in enumerate(tickers):
    ax.scatter(
        individual_volatility[i], 
        expected_returns[i],
        marker='D',
        s=100,
        edgecolors='black',
        linewidth=2,
        zorder=5
    )
    ax.annotate(
        ticker, 
        (individual_volatility[i], expected_returns[i]),
        xytext=(5, 5),
        textcoords='offset points',
        fontsize=9,
        fontweight='bold'
    )

# Plot minimum variance portfolio
ax.scatter(
    min_var['volatility'], 
    min_var['return'],
    marker='*',
    s=400,
    c='blue',
    edgecolors='black',
    linewidth=2,
    label=f"Min Variance (SR={min_var['sharpe_ratio']:.2f})",
    zorder=10
)

# Plot max Sharpe portfolio
ax.scatter(
    max_sharpe['volatility'], 
    max_sharpe['return'],
    marker='*',
    s=400,
    c='gold',
    edgecolors='black',
    linewidth=2,
    label=f"Max Sharpe (SR={max_sharpe['sharpe_ratio']:.2f})",
    zorder=10
)

# Plot Capital Market Line
risk_free_rate = 0.02
x_cml = np.linspace(0, max(efficient_frontier['volatility']) * 1.2, 100)
y_cml = risk_free_rate + max_sharpe['sharpe_ratio'] * x_cml
ax.plot(x_cml, y_cml, 'g--', linewidth=2, label='Capital Market Line')

ax.set_xlabel('Volatility (Standard Deviation)', fontsize=12)
ax.set_ylabel('Expected Return', fontsize=12)
ax.set_title('Markowitz Efficient Frontier', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=10)
ax.set_xlim(0, max(individual_volatility) * 1.3)
ax.set_ylim(min(expected_returns) * 0.8, max(expected_returns) * 1.2)

# Format axes as percentages
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))

plt.tight_layout()
plt.show()

---
## 9. Portfolio Allocation Visualization

In [ ]:
# Visualize portfolio allocations
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Equal weighted
colors = plt.cm.Set3(np.linspace(0, 1, len(tickers)))
axes[0].pie(equal_weights, labels=tickers, autopct='%1.1f%%', colors=colors)
axes[0].set_title(f'Equal Weighted\nReturn: {eq_stats["return"]:.1%}, Vol: {eq_stats["volatility"]:.1%}')

# Minimum variance
min_var_weights = min_var['weights']
min_var_weights_display = np.where(min_var_weights > 0.01, min_var_weights, 0)
labels_mv = [t if w > 0.01 else '' for t, w in zip(tickers, min_var_weights)]
axes[1].pie(min_var_weights_display, labels=labels_mv, autopct=lambda p: f'{p:.1f}%' if p > 1 else '', colors=colors)
axes[1].set_title(f'Minimum Variance\nReturn: {min_var["return"]:.1%}, Vol: {min_var["volatility"]:.1%}')

# Max Sharpe
max_sharpe_weights = max_sharpe['weights']
max_sharpe_weights_display = np.where(max_sharpe_weights > 0.01, max_sharpe_weights, 0)
labels_ms = [t if w > 0.01 else '' for t, w in zip(tickers, max_sharpe_weights)]
axes[2].pie(max_sharpe_weights_display, labels=labels_ms, autopct=lambda p: f'{p:.1f}%' if p > 1 else '', colors=colors)
axes[2].set_title(f'Max Sharpe Ratio\nReturn: {max_sharpe["return"]:.1%}, Vol: {max_sharpe["volatility"]:.1%}')

plt.suptitle('Portfolio Allocation Comparison', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Weight comparison bar chart
weight_comparison = pd.DataFrame({
    'Equal Weighted': equal_weights,
    'Min Variance': min_var['weights'],
    'Max Sharpe': max_sharpe['weights']
}, index=tickers)

ax = weight_comparison.plot(kind='bar', figsize=(12, 6), width=0.8, edgecolor='black')
ax.set_xlabel('Asset', fontsize=12)
ax.set_ylabel('Weight', fontsize=12)
ax.set_title('Portfolio Weight Comparison', fontsize=14, fontweight='bold')
ax.legend(title='Portfolio Type')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nWeight Comparison Table:")
print(weight_comparison.round(4).to_string())

---
## 10. Performance Metrics Summary

In [ ]:
# Comprehensive portfolio comparison
portfolio_comparison = pd.DataFrame({
    'Equal Weighted': [
        eq_stats['return'],
        eq_stats['volatility'],
        eq_stats['sharpe_ratio'],
        eq_stats['return'] / eq_stats['volatility']
    ],
    'Min Variance': [
        min_var['return'],
        min_var['volatility'],
        min_var['sharpe_ratio'],
        min_var['return'] / min_var['volatility']
    ],
    'Max Sharpe': [
        max_sharpe['return'],
        max_sharpe['volatility'],
        max_sharpe['sharpe_ratio'],
        max_sharpe['return'] / max_sharpe['volatility']
    ]
}, index=['Expected Return', 'Volatility', 'Sharpe Ratio', 'Return/Risk'])

print("=" * 60)
print("PORTFOLIO PERFORMANCE COMPARISON")
print("=" * 60)
print(portfolio_comparison.round(4).to_string())

In [ ]:
# Calculate additional risk metrics
def calculate_var(weights, returns, confidence_level=0.95):
    """Calculate Value at Risk (historical method)."""
    portfolio_returns = returns @ weights
    var = np.percentile(portfolio_returns, (1 - confidence_level) * 100)
    return var

def calculate_cvar(weights, returns, confidence_level=0.95):
    """Calculate Conditional Value at Risk (Expected Shortfall)."""
    portfolio_returns = returns @ weights
    var = calculate_var(weights, returns, confidence_level)
    cvar = portfolio_returns[portfolio_returns <= var].mean()
    return cvar

def calculate_max_drawdown(weights, prices):
    """Calculate maximum drawdown."""
    portfolio_value = (prices @ weights)
    portfolio_value = portfolio_value / portfolio_value.iloc[0]  # Normalize
    rolling_max = portfolio_value.expanding().max()
    drawdown = (portfolio_value - rolling_max) / rolling_max
    return drawdown.min()

# Calculate metrics for each portfolio
returns_array = returns.values
prices_df = data.copy()

risk_metrics = pd.DataFrame(index=['VaR (95%)', 'CVaR (95%)', 'Max Drawdown'])

for name, weights in [('Equal Weighted', equal_weights), 
                       ('Min Variance', min_var['weights']),
                       ('Max Sharpe', max_sharpe['weights'])]:
    var = calculate_var(weights, returns_array)
    cvar = calculate_cvar(weights, returns_array)
    max_dd = calculate_max_drawdown(weights, prices_df)
    risk_metrics[name] = [var, cvar, max_dd]

print("\n" + "=" * 60)
print("RISK METRICS (Daily)")
print("=" * 60)
print(risk_metrics.round(4).to_string())

---
## 11. Backtesting the Strategies

In [ ]:
# Backtest portfolio performance
def backtest_portfolio(weights, returns):
    """Calculate cumulative returns for a portfolio."""
    portfolio_returns = returns @ weights
    cumulative_returns = (1 + portfolio_returns).cumprod()
    return cumulative_returns

# Calculate cumulative returns
cumulative_equal = backtest_portfolio(equal_weights, returns)
cumulative_min_var = backtest_portfolio(min_var['weights'], returns)
cumulative_max_sharpe = backtest_portfolio(max_sharpe['weights'], returns)

# Plot cumulative returns
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(cumulative_equal.index, cumulative_equal.values, 
        label='Equal Weighted', linewidth=2)
ax.plot(cumulative_min_var.index, cumulative_min_var.values, 
        label='Min Variance', linewidth=2)
ax.plot(cumulative_max_sharpe.index, cumulative_max_sharpe.values, 
        label='Max Sharpe', linewidth=2)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Cumulative Return', fontsize=12)
ax.set_title('Portfolio Backtest: Cumulative Returns', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.axhline(y=1, color='black', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

# Final returns
print("\nFinal Cumulative Returns:")
print(f"  Equal Weighted: {cumulative_equal.iloc[-1]:.2f}x ({(cumulative_equal.iloc[-1]-1)*100:.1f}%)")
print(f"  Min Variance: {cumulative_min_var.iloc[-1]:.2f}x ({(cumulative_min_var.iloc[-1]-1)*100:.1f}%)")
print(f"  Max Sharpe: {cumulative_max_sharpe.iloc[-1]:.2f}x ({(cumulative_max_sharpe.iloc[-1]-1)*100:.1f}%)")

In [ ]:
# Calculate rolling Sharpe ratio
def rolling_sharpe(weights, returns, window=252, risk_free_rate=0.02):
    """Calculate rolling Sharpe ratio."""
    portfolio_returns = returns @ weights
    rolling_mean = portfolio_returns.rolling(window).mean() * 252
    rolling_std = portfolio_returns.rolling(window).std() * np.sqrt(252)
    rolling_sharpe = (rolling_mean - risk_free_rate) / rolling_std
    return rolling_sharpe

# Calculate rolling metrics
rolling_sharpe_equal = rolling_sharpe(equal_weights, returns)
rolling_sharpe_min_var = rolling_sharpe(min_var['weights'], returns)
rolling_sharpe_max_sharpe = rolling_sharpe(max_sharpe['weights'], returns)

# Plot
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(rolling_sharpe_equal.index, rolling_sharpe_equal.values, 
        label='Equal Weighted', linewidth=1.5, alpha=0.8)
ax.plot(rolling_sharpe_min_var.index, rolling_sharpe_min_var.values, 
        label='Min Variance', linewidth=1.5, alpha=0.8)
ax.plot(rolling_sharpe_max_sharpe.index, rolling_sharpe_max_sharpe.values, 
        label='Max Sharpe', linewidth=1.5, alpha=0.8)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Sharpe Ratio', fontsize=12)
ax.set_title('Rolling 1-Year Sharpe Ratio', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.axhline(y=0, color='black', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

---
## 12. Key Takeaways & Interview Questions

### Key Concepts Learned

1. **Mean-Variance Optimization**: Balances expected return against portfolio risk
2. **Efficient Frontier**: Set of portfolios offering maximum return for each risk level
3. **Minimum Variance Portfolio**: Lowest risk achievable through diversification
4. **Maximum Sharpe Portfolio**: Best risk-adjusted return (tangency portfolio)
5. **Capital Market Line**: Combines risk-free asset with tangency portfolio

### Common Interview Questions

**Q1: What are the assumptions of Markowitz's Mean-Variance framework?**
- Investors are risk-averse and rational
- Returns are normally distributed
- Investors care only about mean and variance
- Markets are frictionless (no transaction costs)
- Single period investment horizon

**Q2: What are the limitations of Mean-Variance Optimization?**
- Sensitive to input estimation errors (especially expected returns)
- Assumes normal distribution (ignores fat tails)
- Can produce extreme/concentrated positions
- Historical returns may not predict future
- Ignores higher moments (skewness, kurtosis)

**Q3: How do you handle estimation error in practice?**
- Shrinkage estimators (Ledoit-Wolf)
- Black-Litterman model
- Robust optimization
- Resampling methods
- Regularization (constraints on weights)

**Q4: Explain the Sharpe Ratio and its limitations.**
- Sharpe = (Return - Rf) / Volatility
- Assumes symmetric distribution
- Penalizes upside volatility equally
- Can be manipulated (smoothing)
- Sortino ratio addresses downside only

In [ ]:
# Summary statistics
print("=" * 70)
print("DAY 01 SUMMARY: MARKOWITZ MEAN-VARIANCE OPTIMIZATION")
print("=" * 70)
print(f"\nAssets analyzed: {', '.join(tickers)}")
print(f"Data period: {returns.index[0].date()} to {returns.index[-1].date()}")
print(f"Number of observations: {len(returns)}")
print("\nOptimal Portfolio Weights (Max Sharpe):")
for ticker, weight in zip(tickers, max_sharpe['weights']):
    if weight > 0.01:
        print(f"  {ticker}: {weight:.1%}")
print(f"\nMax Sharpe Portfolio Performance:")
print(f"  Expected Annual Return: {max_sharpe['return']:.2%}")
print(f"  Annual Volatility: {max_sharpe['volatility']:.2%}")
print(f"  Sharpe Ratio: {max_sharpe['sharpe_ratio']:.3f}")
print("\n" + "=" * 70)

---
## 13. Practice Exercises

### Exercise 1: Different Asset Universe
Re-run the optimization with a different set of assets (e.g., international ETFs, sectors)

### Exercise 2: Short Selling
Allow short selling (`allow_short_selling=True`) and compare the efficient frontier

### Exercise 3: Weight Constraints
Add maximum weight constraints (e.g., no asset > 30%) and observe the impact

### Exercise 4: Rolling Optimization
Implement a rolling window optimization to see how optimal weights change over time

### Exercise 5: Transaction Costs
Include transaction costs in the optimization and measure turnover

In [ ]:
# Exercise starter: Weight constraints example
def maximize_sharpe_with_constraints(expected_returns, cov_matrix, 
                                      max_weight=0.30, min_weight=0.02,
                                      risk_free_rate=0.02):
    """
    Find max Sharpe portfolio with weight constraints.
    """
    n_assets = len(expected_returns)
    
    def neg_sharpe(weights):
        return -portfolio_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate)
    
    init_weights = np.array([1/n_assets] * n_assets)
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    
    # Weight bounds with constraints
    bounds = tuple((min_weight, max_weight) for _ in range(n_assets))
    
    result = minimize(
        fun=neg_sharpe,
        x0=init_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    if result.success:
        weights = result.x
        stats = portfolio_stats(weights, expected_returns, cov_matrix, risk_free_rate)
        return {
            'weights': weights,
            'return': stats['return'],
            'volatility': stats['volatility'],
            'sharpe_ratio': stats['sharpe_ratio']
        }
    return None

# Try with constraints
constrained_portfolio = maximize_sharpe_with_constraints(mu, cov, max_weight=0.25)
if constrained_portfolio:
    print("Constrained Portfolio (max 25% per asset):")
    print(f"  Sharpe Ratio: {constrained_portfolio['sharpe_ratio']:.3f}")
    print(f"  vs Unconstrained: {max_sharpe['sharpe_ratio']:.3f}")